# Datasets and profile preparation using Slab2.0

In [3]:
import os
import sys
import shutil
import openquake.sub as sub

# Get the path of the mbtk subduction module
path = sub.__path__[0]

## 1. Pickle the catalogue
First we pickle the catalogue, that is, we create a compact representation of this dataset that can be read quickly by python.

In [ ]:
from openquake.sub.pickle_catalogue import main

catalog = "./../data/catalogue/Leyte_ISCGEMv10_200km.csv"

main(catalog)

source = "catalogue_ext.p"
destination = "./../data/catalogue/wpg_catalogue_sumatra.p"
os.makedirs(os.path.dirname(destination), exist_ok=True)

# Move the file
shutil.move(source, destination)

os.remove("catalogue_ori.p")

./../data/catalogue/Leyte_ISCGEMv10_200km.csv
Catalogue Attribute OID is not a recognised catalogue key
Catalogue Attribute q is not a recognised catalogue key
Catalogue Attribute eventid is not a recognised catalogue key
Catalogue Attribute strike is not a recognised catalogue key
Catalogue Attribute smajax is not a recognised catalogue key
Catalogue Attribute sminax is not a recognised catalogue key
Catalogue Attribute unc is not a recognised catalogue key
Catalogue Attribute q_1 is not a recognised catalogue key
Catalogue Attribute unc_1 is not a recognised catalogue key
Catalogue Attribute q_12 is not a recognised catalogue key
Catalogue Attribute s is not a recognised catalogue key
Catalogue Attribute mo is not a recognised catalogue key
Catalogue Attribute fac is not a recognised catalogue key
Catalogue Attribute mo_auth is not a recognised catalogue key
Catalogue Attribute mpp is not a recognised catalogue key
Catalogue Attribute mpr is not a recognised catalogue key
Catalogue A

## 2. Create cross-sections
Now we create a file containing the input required to plot a cross section given a trace. We generate a set of traces automatically by sampling it at regular disstances. At each point we take the perpendicular direction to the trench to define the direction of the trace. 

In [ ]:
import openquake.sub.create_multiple_cross_sections as cmcs

profile_path = "./../ini/create_profiles.txt"

cmcs(profile_path)


source = "cs_traces.cs"
destination = "./../model/cs_traces.txt"
os.makedirs(os.path.dirname(destination), exist_ok=True)


!python $path/create_multiple_cross_sections.py ./../ini/create_profiles.txt
!mkdir -p ./../model
!move cs_traces.cs ./../model/cs_traces.txt

Traceback (most recent call last):
  File "c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\.venv\lib\site-packages\openquake\sub\create_multiple_cross_sections.py", line 187, in <module>
    sap.run(main)
  File "c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\.venv\lib\site-packages\openquake\baselib\sap.py", line 212, in run
    return _run(parser(funcdict, **parserkw), argv)
  File "c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\.venv\lib\site-packages\openquake\baselib\sap.py", line 203, in _run
    return func(**dic)
  File "c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\.venv\lib\site-packages\openquake\sub\create_multiple_cross_sections.py", line 178, in main
    cs_dict = get_cs(trench, argv[0], cs_length, cs_depth, interdistance, qual)
NameError: name 'argv' is not defined
The syntax of the command is incorrect.
The system cannot find the file specified.


## 3. Calculate profiles

In [4]:
fname_sl = './../data/subduction/sum_slab2_dep_02.23.18.xyz'
fout = './../model/profiles/'
fname_cs = './../model/cs_traces.txt'

In [ ]:
!mkdir -p $fout
!oq sub create_sections_from_slab.py $fname_sl $fout $fname_cs

import openquake.sub.create_2pt5_model

The syntax of the command is incorrect.
usage: oq [-h] [-v]
          {abort,checksum,check_input,collect_jobs,compare,create_jobs,db,dbserver,dump,engine,export,extract,importcalc,info,ltcsv,mosaic,nrml_from,nrml_to,plot,plot_assets,plot_sites,postzip,prepare_site_model,purge,reaggregate,reduce_exp,reduce_sm,reduce_smlt,renumber_sm,rerun,reset,restore,run,sample,sensitivity_analysis,shakemap2gmfs,shell,show,show_attrs,submit,tidy,to_hdf5,upgrade_nrml,webui,workers,zip}
          ...
oq: error: argument {abort,checksum,check_input,collect_jobs,compare,create_jobs,db,dbserver,dump,engine,export,extract,importcalc,info,ltcsv,mosaic,nrml_from,nrml_to,plot,plot_assets,plot_sites,postzip,prepare_site_model,purge,reaggregate,reduce_exp,reduce_sm,reduce_smlt,renumber_sm,rerun,reset,restore,run,sample,sensitivity_analysis,shakemap2gmfs,shell,show,show_attrs,submit,tidy,to_hdf5,upgrade_nrml,webui,workers,zip}: invalid choice: 'sub' (choose from 'abort', 'checksum', 'check_input', 'collect_jobs'

## 4. Create the top of the slab surface

In [2]:
max_sampling_distace=10.0
UpperSeismoDepth=0
LowerSeismoDepth=250.

input_path='./../model/profiles/'
output_folder='./../model/surfaces/sum'

In [3]:
# !mkdir -p $out_folder
# !oqm sub build_complex_surface $in_folder $sampling $out_folder $uppdep $lowdep


from openquake.sub.build_complex_surface import c_build_complex_surface
c_build_complex_surface(input_path, max_sampling_distace, output_folder, UpperSeismoDepth, LowerSeismoDepth)

11/18/2024 12:15:59 PM Did not find cross-sections in c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\model\profiles
 exiting


SystemExit: 0

c:\Users\USER\Desktop\subduction\oq-mbtk-sub-example\.venv\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 5. Plotting

Now we create a .geojson file containing the profiles and edges which can be visualized with a GIS.

In [8]:
pattern = './../model/profiles/cs*.csv'
output = './../output/geojson/profiles.geojson'

In [10]:
import json
from ipyleaflet import Map, GeoJSON

In [11]:
with open(output, 'r') as f:
    data = json.load(f)
geo_json = GeoJSON(data=data, style={'opacity': 1, 'dashArray': '2', 'fillOpacity': 0.1, 'weight': 1})
m = Map(center=(0.0, 100.0), zoom=5)
m.add_layer(geo_json)
m

FileNotFoundError: [Errno 2] No such file or directory: './../output/geojson/profiles.geojson'

In [6]:
import numpy as np
import h5py
import pickle

# Load OQ tools
from openquake.hmtk.parsers.catalogue import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueWriter

In [7]:
# Configuration files
cat_pickle_filename = '~/model/catalogue/csv/catalogue.pkl'
treg = '~/model/catalogue/classification/classified.hdf5'

# Reading TR hdf5 file and creating the list of tectonic regions
aaa = []
f = h5py.File(treg, "r")
for key in f.keys():
    aaa.append(key)
    alen = len(f[key])
    print(key)
f.close()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '~/model/catalogue/classification/classified.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)